In [ ]:
# Install required packages if not already installed
# %pip install mediapipe opencv-python

import cv2
import numpy as np
import mediapipe as mp
import random
import os
import matplotlib.pyplot as plt

# Choose a random image from mens or womens dataset
def get_random_image():
    mens_dirs = [
        '/Users/rohanrao/Desktop/hair4face/mens faceshape/testing_set/oblong',
        '/Users/rohanrao/Desktop/hair4face/mens faceshape/testing_set/oval',
        '/Users/rohanrao/Desktop/hair4face/mens faceshape/testing_set/round',
        '/Users/rohanrao/Desktop/hair4face/mens faceshape/testing_set/square'
    ]
    womens_dirs = [
        '/Users/rohanrao/Desktop/hair4face/womens faceshape/testing_set/Heart',
        '/Users/rohanrao/Desktop/hair4face/womens faceshape/testing_set/Oblong',
        '/Users/rohanrao/Desktop/hair4face/womens faceshape/testing_set/Oval',
        '/Users/rohanrao/Desktop/hair4face/womens faceshape/testing_set/Round',
        '/Users/rohanrao/Desktop/hair4face/womens faceshape/testing_set/Square'
    ]
    all_dirs = mens_dirs + womens_dirs
    chosen_dir = random.choice(all_dirs)
    images = [f for f in os.listdir(chosen_dir) if f.lower().endswith('.jpg')]
    if not images:
        raise FileNotFoundError(f'No images found in {chosen_dir}')
    chosen_img = random.choice(images)
    return os.path.join(chosen_dir, chosen_img)

image_path = get_random_image()
print(f'Using image: {image_path}')

FOREHEAD_FACTOR = 0.8
mp_face_mesh = mp.solutions.face_mesh

def to_px(lm, w, h):
    return np.array([lm.x * w, lm.y * h])

def euclid(a, b):
    return float(np.linalg.norm(a - b))

def compute_forehead(landmarks, w, h, method="extrapolate", factor=0.35):
    L_EYE_IN = 133
    R_EYE_IN = 362
    CHIN = 152
    if method == "landmark10":
        return to_px(landmarks[10], w, h)
    if method == "topmost":
        ys = [lm.y for lm in landmarks]
        idx_min = int(np.argmin(ys))
        return to_px(landmarks[idx_min], w, h)
    mid_eye = (to_px(landmarks[L_EYE_IN], w, h) + to_px(landmarks[R_EYE_IN], w, h)) / 2.0
    chin = to_px(landmarks[CHIN], w, h)
    direction = mid_eye - chin
    forehead = mid_eye + direction * factor
    forehead[0] = float(np.clip(forehead[0], 0, w-1))
    forehead[1] = float(np.clip(forehead[1], 0, h-1))
    return forehead

img = cv2.imread(image_path)
if img is None:
    raise FileNotFoundError(f"Could not open image at {image_path}")
h, w = img.shape[:2]
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

with mp_face_mesh.FaceMesh(static_image_mode=True,
                           max_num_faces=1,
                           refine_landmarks=True,
                           min_detection_confidence=0.5) as fm:
    res = fm.process(img_rgb)
    if not res.multi_face_landmarks:
        raise RuntimeError("No face detected")
    lm = res.multi_face_landmarks[0].landmark
    sel = {
        "left_eye_inner": 133,
        "right_eye_inner": 362,
        "nose_left": 49,
        "nose_right": 279,
        "nose_base": 2,
        "chin": 152,
        "left_face": 234,
        "right_face": 454
    }
    pts = {k: to_px(lm[idx], w, h) for k, idx in sel.items()}
    forehead_extrap = compute_forehead(lm, w, h, method="extrapolate", factor=FOREHEAD_FACTOR)
    forehead_index10 = compute_forehead(lm, w, h, method="landmark10")
    forehead_topmost = compute_forehead(lm, w, h, method="topmost")
    face_width = euclid(pts["left_face"], pts["right_face"])
    face_length = euclid(forehead_extrap, pts["chin"])
    interocular = euclid(pts["left_eye_inner"], pts["right_eye_inner"])
    nose_width = euclid(pts["nose_left"], pts["nose_right"])
    out = img.copy()
    for name, p in pts.items():
        cv2.circle(out, (int(p[0]), int(p[1])), 5, (0,0,255), -1)
        cv2.putText(out, name, (int(p[0])+6, int(p[1])-6), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,0,255), 1)
    cv2.circle(out, tuple(forehead_extrap.astype(int)), 7, (0,255,0), -1)
    cv2.putText(out, "forehead_extrap", (int(forehead_extrap[0])+6, int(forehead_extrap[1])-6),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0,160,0), 1)
    cv2.circle(out, tuple(forehead_index10.astype(int)), 5, (255,0,0), -1)
    cv2.putText(out, "landmark[10]", (int(forehead_index10[0])+6, int(forehead_index10[1])-6),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, (200,0,0), 1)
    cv2.circle(out, tuple(forehead_topmost.astype(int)), 5, (0,255,255), -1)
    cv2.putText(out, "topmost", (int(forehead_topmost[0])+6, int(forehead_topmost[1])-6),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0,160,160), 1)
    cv2.line(out, tuple(pts["left_face"].astype(int)), tuple(pts["right_face"].astype(int)), (255,0,0), 1)
    cv2.line(out, tuple(pts["left_eye_inner"].astype(int)), tuple(pts["right_eye_inner"].astype(int)), (255,0,0), 1)
    cv2.line(out, tuple(forehead_extrap.astype(int)), tuple(pts["chin"].astype(int)), (255,0,0), 1)
    plt.figure(figsize=(8,8))
    plt.imshow(cv2.cvtColor(out, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()
    print("Face measurements (px):")
    print(f" face_width:  {face_width:.1f}")
    print(f" face_length: {face_length:.1f}  (extrapolated forehead -> chin)")
    print(f" interocular: {interocular:.1f}")
    print(f" nose_width:  {nose_width:.1f}")
    print()
    print("Forehead points (px):")
    print(f" extrapolated: ({forehead_extrap[0]:.1f}, {forehead_extrap[1]:.1f})  factor={FOREHEAD_FACTOR}")
    print(f" landmark[10]:  ({forehead_index10[0]:.1f}, {forehead_index10[1]:.1f})")
    print(f" topmost:       ({forehead_topmost[0]:.1f}, {forehead_topmost[1]:.1f})")
    print(f" distance landmark[10 ↔ extrapolated]: {euclid(forehead_index10, forehead_extrap):.1f} px")
    print(f" distance topmost ↔ extrapolated:       {euclid(forehead_topmost, forehead_extrap):.1f} px")

ModuleNotFoundError: No module named 'cv2'

In [ ]:
# --- Classification thresholds and logic ---

def classify_interocular(interocular, face_width):
    ratio = interocular / face_width
    if ratio < 0.24: return "narrow"
    elif ratio <= 0.30: return "average"
    else: return "wide"

def classify_nose_width(nose_width, face_width):
    ratio = nose_width / face_width
    if ratio < 0.18: return "narrow"
    elif ratio <= 0.20: return "average"
    else: return "wide"

def classify_face_shape(face_length, face_width, thirds):
    ratio = face_length / face_width
    upper, middle, lower = thirds
    if 1.3 <= ratio <= 1.45 and all(0.28 <= t <= 0.38 for t in thirds):
        return "oval"
    if ratio < 1.25:
        return "round"
    if ratio > 1.55:
        return "oblong"
    if abs(lower - upper) < 0.05 and 1.25 <= ratio <= 1.5:
        return "square"
    if upper > 0.37:
        return "heart"
    return "oval"

def hairstyle_suggestions(shape):
    suggestions = {
        "oval": ["versatile styles", "soft layers", "side-part"],
        "round": ["volume at crown", "longer layers", "avoid blunt cuts"],
        "square": ["soft layers", "side-swept fringe", "avoid flat cuts"],
        "oblong": ["shorter top, fuller sides", "side bangs", "medium-length layers"],
        "heart": ["soft layers around jaw", "side-swept bangs", "avoid height on top"],
    }
    return suggestions.get(shape, ["balanced cuts", "natural styles"])

# --- Compute facial thirds (corrected) ---
# Use glabella (landmark 9) as brow baseline
upper_third = abs(pts["brow_glabella"][1] - forehead_extrap[1])
middle_third = abs(pts["nose_base"][1] - pts["brow_glabella"][1])
lower_third = abs(pts["chin"][1] - pts["nose_base"][1])

total_length = face_length
thirds = (upper_third/total_length, middle_third/total_length, lower_third/total_length)

# --- Classify ---
interocular_cat = classify_interocular(interocular, face_width)
nose_cat = classify_nose_width(nose_width, face_width)
face_shape = classify_face_shape(face_length, face_width, thirds)
haircuts = hairstyle_suggestions(face_shape)

# --- Print results ---
print("\nClassification:")
print(f" interocular distance: {interocular_cat}")
print(f" nose width: {nose_cat}")
print(f" facial thirds ratio: upper={thirds[0]:.2f}, middle={thirds[1]:.2f}, lower={thirds[2]:.2f}")
print(f" predicted face shape: {face_shape}")
print(" hairstyle suggestions:")
for s in haircuts:
    print("  -", s)


In [ ]:

import cv2
import mediapipe as mp
import numpy as np

# Constants
FOREHEAD_FACTOR = 0.65
mp_face_mesh = mp.solutions.face_mesh

# Initialize FaceMesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True)

def compute_facial_thirds(image_path):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Could not read image: {image_path}")
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb)
    if not results.multi_face_landmarks:
        print("No face detected.")
        return image
    
    h, w, _ = image.shape
    for face_landmarks in results.multi_face_landmarks:
        # Facial landmarks
        brow_y = np.mean([face_landmarks.landmark[105].y, face_landmarks.landmark[334].y]) * h
        nose_base_y = face_landmarks.landmark[2].y * h
        philtrum_y = face_landmarks.landmark[0].y * h
        chin_y = face_landmarks.landmark[152].y * h

        # Estimate forehead
        forehead_y = max(0, brow_y - (nose_base_y - brow_y) * FOREHEAD_FACTOR)

        # Calculate facial thirds
        top_third = brow_y - forehead_y
        middle_third = nose_base_y - brow_y
        bottom_third = chin_y - philtrum_y

        total_height = chin_y - forehead_y
        ratios = [top_third / total_height, middle_third / total_height, bottom_third / total_height]

        # Draw guide lines
        for y_pos, label in zip([forehead_y, brow_y, nose_base_y, philtrum_y, chin_y],
                                ["Forehead (est)", "Brows", "Nose Base", "Philtrum", "Chin"]):
            cv2.line(image, (0, int(y_pos)), (w, int(y_pos)), (0, 255, 0), 1)
            cv2.putText(image, label, (10, int(y_pos)-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        # Print numeric results
        print(f"Top third (forehead→brows): {top_third:.2f}px")
        print(f"Middle third (brows→nose): {middle_third:.2f}px")
        print(f"Bottom third (philtrum→chin): {bottom_third:.2f}px")
        print(f"Ratios (normalized): {ratios}")

    return image

# Example usage:
# img = compute_facial_thirds('face.jpg')
# cv2.imshow('Facial Thirds', img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


### 📘 Facial Ratio and Shape Estimation using Mediapipe

In [ ]:

# Facial metrics analysis using Mediapipe FaceMesh

import mediapipe as mp
import cv2
import numpy as np

mp_face_mesh = mp.solutions.face_mesh

# Define key landmark indices for upper, middle, and lower thirds
# Reference: Mediapipe FaceMesh (468 points)
FACIAL_THIRD_POINTS = {
    "forehead": 10,        # near top of forehead
    "brow_ridge": 9,       # glabella region
    "philtrum": 0,         # base of the nose / philtrum
    "chin": 152            # lowest point on the chin
}

def calculate_facial_thirds(image_path):
    """
    Calculates normalized facial thirds ratios using Mediapipe landmarks.
    Returns dict with ratio values and classification.
    """
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Invalid image path")

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    with mp_face_mesh.FaceMesh(static_image_mode=True, refine_landmarks=True) as face_mesh:
        results = face_mesh.process(image_rgb)

    if not results.multi_face_landmarks:
        raise ValueError("No face detected")

    landmarks = results.multi_face_landmarks[0].landmark

    # Extract vertical positions (y coordinates)
    forehead_y = landmarks[FACIAL_THIRD_POINTS["forehead"]].y
    brow_y = landmarks[FACIAL_THIRD_POINTS["brow_ridge"]].y
    philtrum_y = landmarks[FACIAL_THIRD_POINTS["philtrum"]].y
    chin_y = landmarks[FACIAL_THIRD_POINTS["chin"]].y

    # Compute distances (normalized)
    total_length = chin_y - forehead_y
    upper = (brow_y - forehead_y) / total_length
    middle = (philtrum_y - brow_y) / total_length
    lower = (chin_y - philtrum_y) / total_length

    # Classification logic
    if abs(upper - lower) < 0.05 and abs(middle - lower) < 0.05:
        shape = "Oval"
    elif lower > upper and lower > middle:
        shape = "Round"
    else:
        shape = "Long"

    # Hairstyle suggestions
    hairstyle_tips = []
    if shape == "Round":
        hairstyle_tips = ["Add volume at the crown", "Try longer layers", "Avoid blunt cuts"]
    elif shape == "Oval":
        hairstyle_tips = ["Try side-swept bangs", "Soft waves work well", "Most styles suit oval faces"]
    else:
        hairstyle_tips = ["Avoid flat styles", "Add width with waves", "Layer around jawline"]

    return {
        "facial_thirds_ratio": {
            "upper": round(upper, 2),
            "middle": round(middle, 2),
            "lower": round(lower, 2)
        },
        "predicted_face_shape": shape,
        "hairstyle_suggestions": hairstyle_tips
    }

# Example usage:
# result = calculate_facial_thirds("face.jpg")
# print(result)
